# ETL del catálogo de la SSPM

## Generación del catálogo

### Setup

In [2]:
import generate_catalog
import json
import pandas as pd
import os
from openpyxl import load_workbook

In [3]:
#completar con lo que corresponda
catalog_xlsx_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/catalogo-sspm.xlsx"
catalog_json_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"
REPORTES_DIR = os.path.join("/Users/abenassi/github/series-tiempo", "catalogo", "datos", "reportes")

In [35]:
def find_ws_name(wb, name):
    if type(wb) == str or type(wb) == unicode:
        wb = load_workbook(wb, read_only=True, data_only=True)
    
    for sheetname in wb.sheetnames:
        if sheetname.lower() == name.lower():
            return sheetname
    
    return None

In [36]:
field_cols = [
    "field_title", "field_id", "field_identifierCell", "field_dataStartCell",
    "dataset_identifier",     "dataset_title",
    "distribution_identifier", "distribution_title"
]
distribution_cols = [
    "distribution_iedFileURL", "distribution_iedFileSheet", "distribution_identifier",
    "distribution_title", "dataset_identifier",     "dataset_title"
]

In [14]:
df_distribution = pd.read_excel(catalog_xlsx_path, find_ws_name(catalog_xlsx_path, "distribution"))[distribution_cols]
df_field = pd.read_excel(catalog_xlsx_path, find_ws_name(catalog_xlsx_path, "field"))[field_cols]

In [37]:
merged = df_field[field_cols].merge(
    df_distribution[distribution_cols], how="outer", indicator=True, on=[
        "dataset_identifier", "dataset_title", "distribution_identifier", "distribution_title"]
)

In [39]:
merged[merged['_merge'] == 'left_only'].to_excel(
        os.path.join(REPORTES_DIR, "reporte-fields-sin-distribution.xlsx"),
        encoding="utf-8", index=False)
merged[merged['_merge'] == 'right_only'].to_excel(
        os.path.join(REPORTES_DIR, "reporte-distribution-sin-fields.xlsx"),
        encoding="utf-8", index=False)

In [40]:
df_etl_params = merged[merged['_merge'] == 'both']

### Main steps

In [26]:
# leo catalogo en excel
catalog_xlsx = generate_catalog.read_xlsx_catalog(catalog_xlsx_path)

# escribo catalogo en json
generate_catalog.write_json_catalog(catalog, catalog_json_path)

# genero reportes de validación y catálogo filtrado
catalog_filtered = generate_catalog.validate_and_filter(catalog)

# escribo catalogo en json
generate_catalog.write_json_catalog(catalog_filtered, catalog_json_path)


Se encontro un dataset con ID '2', pero su titulo es 'Oferta y Demanda Globales [Base 1993]' en lugar del
esperado '#NAME?'. Este dataset NO se considerara.
No hay ningun dataset con el identifier 2
No se encontro el dataset '#NAME?' especificado para el campo
'demanda_global_ibif_equipo_durable_produccion_material_transporte_nacional' (fila #30 de la hoja "Field"). Este campo no figurara en el data.json de salida.

Se encontro un dataset con ID '4', pero su titulo es 'Oferta y Demanda Globales [Base 2004]' en lugar del
esperado '#NAME?'. Este dataset NO se considerara.
No hay ningun dataset con el identifier 4
No se encontro el dataset '#NAME?' especificado para el campo
'demanda_global_total' (fila #130 de la hoja "Field"). Este campo no figurara en el data.json de salida.


### Pruebas y chequeos

In [37]:
catalog_sspm = generate_catalog.readers.read_catalog(catalog_json_path)[0]

In [42]:
len(catalog_sspm["dataset"]), len(catalog["dataset"]), len(catalog_filtered[0]["dataset"])

(34, 34, 34)

In [27]:
type(catalog), type(catalog_filtered)

(dict, list)

## Scrape datasets IED

### Setup

In [4]:
import os
from xlseries import XlSeries
import pandas as pd
import pydatajson.readers
from copy import deepcopy
from scrape_datasets import scrape_distribution, get_distribution_metadata, get_dataset_metadata, scrape_dataset, scrape_file, get_field_metadata

In [5]:
catalog_json_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"
etl_params_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/etl_params.csv"
ied_data_dir = "/Users/abenassi/github/series-tiempo/catalogo/datos/ied/"

In [6]:
DATASETS_DIR = "/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/"
REPORTES_DIR = "/Users/abenassi/github/series-tiempo/catalogo/datos/reportes/"

### Main steps

In [5]:
ied_xlsx_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/ied/apendice4.xlsx"

In [7]:
etl_params = pd.read_csv(etl_params_path, dtype={"distribution_identifier": "str"})

In [8]:
catalog = pydatajson.readers.read_catalog(catalog_json_path)

In [9]:
%time report_datasets, report_distributions = scrape_file(ied_xlsx_path, etl_params, catalog, replace=False)

Distribución 1.1: OK (Skipped)
Distribución 1.2: OK (Skipped)
Distribución 2.1: OK (Skipped)
Distribución 2.2: OK (Skipped)
Distribución 2.3: OK (Skipped)
Distribución 2.4: OK (Skipped)
Distribución 3.1: OK (Skipped)
Distribución 3.2: ERROR (AssertionError('End must be greater than start!',))
Distribución 4.1: OK (Skipped)
Distribución 4.2: OK (Skipped)
Distribución 4.3: OK (Skipped)
Distribución 4.4: OK (Skipped)
Distribución 5.1: ERROR (FieldTitleRepetitionError(u"Hay mas de 1 field con title indice_tiempo: [{u'dataStartCell': u'A7', u'title': u'indice_tiempo', u'specialTypeDetail': u'R/P1Y', u'specialType': u'time_index', u'identifierCell': u'A7', u'type': u'date', u'id': u'5.1_IT_1993_ A_13'}, {u'dataStartCell': u'A8', u'title': u'indice_tiempo', u'specialTypeDetail': u'R/P3M', u'specialType': u'time_index', u'identifierCell': u'A7', u'type': u'date', u'id': u'5.1_IT_1993_ T_13'}]",))
Distribución 5.2: ERROR (IndexError('tuple index out of range',))
Distribución 6.1: OK (Skipped)
D

In [10]:
report_distributions.groupby("distribution_status")[["distribution_identifier"]].count()

,distribution_identifier
distribution_status,
ERROR,33
OK,52


In [11]:
cols_rep_dataset = ["distribution_iedFileURL", "dataset_identifier", "dataset_status"]
report_datasets[cols_rep_dataset].to_csv(os.path.join(REPORTES_DIR, "reporte-datasets-scraping.csv"), 
                       encoding="utf-8", index=False)

cols_rep_distribution = ["distribution_iedFileURL", "dataset_identifier", 
                         "distribution_identifier", "distribution_status", "distribution_notes"]
report_distributions[cols_rep_distribution].to_csv(os.path.join(REPORTES_DIR, "reporte-distributions-scraping.csv"), 
                            encoding="utf-8", index=False)

### Pruebas y chequeos

In [5]:
# cargo el excel de ied
xl = XlSeries(ied_xlsx_path)
ied_xlsx_filename = os.path.basename(ied_xlsx_path)

NameError: name 'ied_xlsx_path' is not defined

In [10]:
distribution = scrape_distribution(xl, etl_params, catalog, "106.2")

AssertionError: No end could be estimated! End: None   | Start: 36

In [ ]:
df_distrib = etl_params[
        etl_params.distribution_identifier == "3.2"
    ]

In [ ]:
df_distrib

In [42]:
result = scrape_dataset(xl, etl_params, catalog, "2", debug_mode=True)

In [48]:
result

"AssertionError('End must be greater than start!',)"

In [19]:
distribution.to_csv("/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/1/63.csv", 
                    encoding="utf-8-sig", index_label="indice_tiempo")

In [17]:
distrib_meta = deepcopy(get_distribution_metadata(catalog, "106.2"))

In [18]:
distrib_meta

{u'accessURL': u'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
 u'description': u'\xcdndice de Precios Internos al por Mayor. Base 1993. Valores trimestrales.',
 u'field': [{u'dataStartCell': u'A35',
   u'id': u'106.2_IT_1993_T_13',
   u'identifierCell': u'A34',
   u'specialType': u'time_index',
   u'specialTypeDetail': u'R/P3M',
   u'title': u'indice_tiempo',
   u'type': u'date'},
  {u'dataStartCell': u'B35',
   u'description': u'IPIM Nivel general. Base 1993. Trimestral',
   u'id': u'106.2_I1NG_1993_T_23',
   u'identifierCell': u'B34',
   u'specialTypeDetail': u'',
   u'title': u'ipim_1993_nivel_general',
   u'type': u'number',
   u'units': u'N\xfamero \xedndice'},
  {u'dataStartCell': u'C35',
   u'description': u'IPIM Total productos nacionales. Base 1993. Trimestral',
   u'id': u'106.2_I1TPN_1993_T_36',
   u'identifierCell': u'C34',
   u'specialTypeDetail': u'',
   u'title': u'ipim_1993_total_productos_nacionales',
   u'type': u'number'

In [22]:
pd.DataFrame(get_distribution_metadata(catalog, "106.2")).iloc[0,:]["iedFileURL"]

u'http://www.economia.gob.ar/download/infoeco/apendice4.xlsx'

In [32]:
etl_params[etl_params.distribution_identifier == "106.2"]

,distribution_iedFileURL,distribution_iedFileSheet,distribution_timeIndexCell,distribution_identifier,distribution_title,field_title,field_id,field_identifierCell,field_dataStartCell
4695,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,indice_tiempo,106.2_IT_1993_T_13,A34,A35
4696,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_nivel_general,106.2_I1NG_1993_T_23,B34,B35
4697,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_total_productos_nacionales,106.2_I1TPN_1993_T_36,C34,C35
4698,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_productos_nacionales_primarios,106.2_I1PNP_1993_T_40,D34,D35
4699,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_productos_nacionales_manufacturados_...,106.2_I1PNMEE_1993_T_63,E34,E35
4700,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_productos_importados,106.2_IPI_1993_T_25,F34,F35
4701,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_precios_relativos_nacionales_importados,106.2_I1PRNI_1993_T_49,G34,G35
4702,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_precios_relativos_primarios_manufact...,106.2_I1PRPMEE_1993_T_70,H34,H35
4703,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A36,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_nivel_general_variacion_mensual,106.2_I1NGVM_1993_T_41,J34,J36
4704,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A39,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_nivel_general_variacion_anual,106.2_I1NGVA_1993_T_39,K34,K39


In [27]:
params = gen_distribution_params(etl_params, catalog, "106.2")
dist_meta = get_distribution_metadata(catalog, "106.2")

NameError: name 'gen_distribution_params' is not defined

In [23]:
params

{u'data_starts': [9, 9, 9, 9, 9, 9],
 u'frequency': u'Y',
 u'headers_coord': ['B8', 'C8', 'D8', 'E8', 'F8', 'G8'],
 u'series_names': ['oferta_global_pbi',
  'oferta_global_importacion',
  'demanda_global_exportacion',
  'demanda_global_ibif',
  'demanda_global_consumo_priv',
  'demanda_global_consumo_publico'],
 u'time_header_coord': 'A8',
 u'worksheet': '1.2 OyD real s.e.'}

In [24]:
dist_meta

{u'accessURL': u'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
 u'description': u'Oferta y Demanda Globales por componente, a precios de comprador, en millones de pesos de 1993 y valores anuales desestacionalizados.',
 u'field': [{u'dataStartCell': u'A9',
   u'id': u'1.1_IT_ D_1993_ A_13',
   u'identifierCell': u'A8',
   u'specialType': u'time_index',
   u'specialTypeDetail': u'R/P1Y',
   u'title': u'indice_tiempo',
   u'type': u'date'},
  {u'dataStartCell': u'B9',
   u'description': u'PIB desestacionalizado, en millones de pesos de 1993 y valores anuales',
   u'id': u'1.1_OGP_ D_1993_ A_17',
   u'identifierCell': u'B8',
   u'title': u'oferta_global_pbi',
   u'type': u'number',
   u'units': u'Millones de pesos, a precios de 1993'},
  {u'dataStartCell': u'C9',
   u'description': u'Importaciones desestacionalizadas, en millones de pesos de 1993 y valores anuales',
   u'id': u'1.1_OGI_ D_1993_ A_25',
   u'identifierCell': u'C8',
   u'title': u

In [6]:
from pydatajson import readers

In [7]:
catalog = readers.read_catalog(catalog_json_path)

In [15]:
for dataset in catalog["dataset"]:
    for distribution in dataset["distribution"]:
        if "field" in distribution:
            for field in distribution["field"]:
                if field["id"] == "1.1_OGP_D_1993_A_17":
                    print dataset["identifier"], distribution["identifier"], field["id"]

1 1.1 1.1_OGP_D_1993_A_17


In [6]:
elecciones = [dataset for dataset in catalog["dataset"] if dataset["title"] == "Resultados Electorales 2015"][0]

In [9]:
for i in [distribution["downloadURL"] for distribution in elecciones["distribution"]]:
    print i

http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PRES_Y_VICE_.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PARLAMENTARIO_NACIONAL.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PARLAMENTARIO_REGIONAL.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/DIP_NAC.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/FMESSN_0313.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/DIP_PROV.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/SEN_PROV.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/GOBERNADOR.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/MUNICIPAL.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/DEPARTAMENTOS.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/FPARTIDOS.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PROVINCIAS.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PV_0101.csv
http://

In [7]:
def get_time_series(
    dataset_id, distribution_id, field_title, 
    fmt="dataframe", time_index="indice_tiempo", field_id=None,
    datasets_dir="/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/"):
    """Devuelve un dataframe con una serie de tiempo."""
    
    distribution_path = os.path.join(datasets_dir, dataset_id, distribution_id + ".csv")
    
    df = pd.read_csv(distribution_path, index_col=time_index, parse_dates=True)[[field_title]]
    if field_id:
        df = df.rename(columns={field_title: field_id})
    
    if fmt=="dataframe":
        return df
    
    elif fmt=="dict" or "list":
        series_dict = json.loads(df.to_json(orient="columns", date_format="iso"))
        if fmt=="dict":
            return series_dict
        else:
            return sorted(series_dict.iteritems(), key=lambda x: x[0])
    
    else:
        raise Exception("No se reconoce el formato")
        

In [8]:
def get_time_series_by_id(field_ids):
    
    if not isinstance(field_ids, list):
        field_ids = [field_ids]
    
    series_params = get_time_series_params(field_ids)
    
    
    

In [9]:
def get_time_series_params(field_ids, catalog="/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"):
    catalog = readers.read_catalog(catalog)
    
    # busca los ids de dataset y distribucion de la serie
    series_params = []
    for dataset in catalog["dataset"]:
        for distribution in dataset["distribution"]:
            if "field" in distribution:
                for field in distribution["field"]:
                    if field["id"] in field_ids:
                        
                        series_params.append((
                            dataset["identifier"],
                            distribution["identifier"],
                            field["title"]
                        ))
    
    return series_params

In [10]:
def get_time_series_df(
    field_ids, use_id=False,
    catalog="/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"):
    catalog = readers.read_catalog(catalog)
    
    if not isinstance(field_ids, list):
        if isinstance(field_ids, dict):
            field_ids = field_ids.keys()
        else:
            field_ids = [field_ids]
    
    series_params = get_time_series_params(field_ids)
    assert len(series_params) > 0, "{} no están en la metadata".format(
    field_ids)
    
    # toma las series y genera un data frame concatenado
    if use_id:
        time_series = [get_time_series(*serie_params, field_id=field_id) 
                       for field_id, serie_params in zip(field_ids, series_params)]
    else:
        time_series = [get_time_series(*serie_params) 
                       for serie_params in series_params]

    return pd.concat(time_series, axis=1)

In [11]:
df = get_time_series_df([
    "4.2_OGP_2004_T_17", "23.2_PIBPM_2004_T_38",
], use_id=True)

NameError: global name 'readers' is not defined

In [11]:
from pydatajson.helpers import parse_repeating_time_interval as parse_freq

In [12]:
def get_time_series_data(
    field_ids,
    catalog_path="/Users/abenassi/github/series-tiempo/catalogo/datos/data.json",
    export_path="/Users/abenassi/github/series-tiempo/catalogo/datos/dumps/tablero-ministerial-ied.csv"):

    if isinstance(field_ids, list):
        field_ids = {field_id: None for field_id in field_ids}
    
    catalog = read_catalog(catalog_path)
    df = get_time_series_df(field_ids.keys(), use_id=True)

    rows = []
    for field_id, data in df.to_dict().items():
        distribution_identifier = field_id.split("_")[0]
        field_metadata = get_field_metadata(catalog, distribution_identifier, 
                                            field_id)
        distribution_metadata = get_distribution_metadata(
            catalog, distribution_identifier)
        
        time_index_freq = filter(
            lambda x: x.get("specialType") == "time_index", 
            distribution_metadata["field"])[0]["specialTypeDetail"]
        
        for field_time, field_value in data.items():
            row = {
                "field_id": field_id, 
                "field_time": field_time, 
                "field_value": field_value,
                "field_title": field_metadata["title"],
                "field_units": field_metadata["units"],
                "field_description": field_metadata["description"],
                "field_short_description": field_ids.get(field_id),
                "distribution_time_index_freq": parse_freq(
                    time_index_freq, to="string")
            }
            rows.append(row)
    
    cols = [
        "field_id", "field_time", "field_value", 
        "field_short_description", "field_units", "field_title",
        "field_description", "distribution_time_index_freq"
    ]
    df = pd.DataFrame(rows)[cols]
    
    if export_path:
        df.to_csv(export_path, encoding="utf8", index=False)
    
    return df
    

In [13]:
tablero_ministerial_ids = {
    "4.2_OGP_2004_T_17": "PBI (base 2004)",
    "45.2_ECTDT_0_T_33": "Tasa de Desempleo (EPH continua)",
    "11.3_ISOM_2004_M_29": "EMAE (base 2004)",
    "63.1_T_0_0_5": "Pobreza (EPH puntual y continua)",
    "66.2_SP_E_0_T_14": "Puestos de trabajo declarados (sector privado)",
    "92.1_TCV_0_0_21": "Tipo de cambio (valuación de reservas BCRA)",
    "92.1_RID_0_0_32": "Reservas internacionales (BCRA)",
    "89.1_TIB_0_0_20": "BADLAR (BCRA)",
    "89.1_TIPF35D_0_0_35": "Plazo fijo 30 a 59 días (BCRA)"
}

In [14]:
from pydatajson.readers import read_catalog
import pydatajson.readers as readers

In [18]:
df = get_time_series_data(tablero_ministerial_ids)

In [19]:
df

,field_id,field_time,field_value,field_short_description,field_units,field_title,field_description,distribution_time_index_freq
0,45.2_ECTDT_0_T_33,2003-02-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
1,45.2_ECTDT_0_T_33,1996-04-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
2,45.2_ECTDT_0_T_33,2010-06-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
3,45.2_ECTDT_0_T_33,1999-04-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
4,45.2_ECTDT_0_T_33,2005-04-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
5,45.2_ECTDT_0_T_33,2013-10-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
6,45.2_ECTDT_0_T_33,2000-03-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
7,45.2_ECTDT_0_T_33,2008-09-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
8,45.2_ECTDT_0_T_33,2013-03-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente
9,45.2_ECTDT_0_T_33,2016-08-01,NaN,Tasa de Desempleo (EPH continua),Porcentaje,eph_continua_tasa_desempleo_total,Tasa de desempleo total. En porcentaje.,Trimestralmente


In [114]:
field_id = "1.1_OGP_D_1993_A_17"
distribution_identifier = field_id.split("_")[0]
field_metadata = get_field_metadata(catalog, distribution_identifier, 
                                            field_id)

In [116]:
distribution_metadata = get_distribution_metadata(catalog, distribution_identifier)

In [119]:
distribution_metadata

{u'accessURL': u'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
 u'description': u'Oferta y Demanda Globales por componente, a precios de comprador, en millones de pesos de 1993 y valores anuales desestacionalizados.',
 u'field': [{u'dataStartCell': u'A9',
   u'id': u'1.1_IT_D_1993_A_13',
   u'identifierCell': u'A8',
   u'specialType': u'time_index',
   u'specialTypeDetail': u'R/P1Y',
   u'title': u'indice_tiempo',
   u'type': u'date'},
  {u'dataStartCell': u'B9',
   u'description': u'PIB desestacionalizado, en millones de pesos de 1993 y valores trimestrales',
   u'id': u'1.1_OGP_D_1993_A_17',
   u'identifierCell': u'B8',
   u'title': u'oferta_global_pbi',
   u'type': u'number',
   u'units': u'Millones de pesos a precios de 1993'},
  {u'dataStartCell': u'C9',
   u'description': u'Importaciones desestacionalizadas, en millones de pesos de 1993 y valores trimestrales',
   u'id': u'1.1_OGI_D_1993_A_25',
   u'identifierCell': u'C8',
   u'title'

u'R/P1Y'

In [17]:
def get_time_series_dict(field_ids):
    
    if not isinstance(field_ids, list):
        field_ids = [field_ids]
    
    series_params = get_time_series_params(field_ids)
    
    time_series = {
        field_id: get_time_series(*serie_params, fmt="list")
        for field_id, serie_params in zip(field_ids, series_params)
    }
    
    return time_series

In [18]:
ied_ts_ids = [
    "11.3_VIPAA_2004_M_31", # EMAE var mensual interanual
    "45.2_ECTDT_0_T_33", # Desempleo trimestral
    "89.1_TIB_0_0_20", # BADLAR mensual
    "115.3_TCRM_0_M_29", # TCRM mensual
]

In [22]:
ts_dict = get_time_series_dict(ied_ts_ids)

In [23]:
ts_dict

{'11.3_VIPAA_2004_M_31': [(u'var_igual_periodo_anio_anterior',
   {u'2004-01-01T00:00:00.000Z': None,
    u'2004-02-01T00:00:00.000Z': None,
    u'2004-03-01T00:00:00.000Z': None,
    u'2004-04-01T00:00:00.000Z': None,
    u'2004-05-01T00:00:00.000Z': None,
    u'2004-06-01T00:00:00.000Z': None,
    u'2004-07-01T00:00:00.000Z': None,
    u'2004-08-01T00:00:00.000Z': None,
    u'2004-09-01T00:00:00.000Z': None,
    u'2004-10-01T00:00:00.000Z': None,
    u'2004-11-01T00:00:00.000Z': None,
    u'2004-12-01T00:00:00.000Z': None,
    u'2005-01-01T00:00:00.000Z': 0.0631114207,
    u'2005-02-01T00:00:00.000Z': 0.0657826605,
    u'2005-03-01T00:00:00.000Z': 0.0861095782,
    u'2005-04-01T00:00:00.000Z': 0.1362787407,
    u'2005-05-01T00:00:00.000Z': 0.1527559219,
    u'2005-06-01T00:00:00.000Z': 0.1025891554,
    u'2005-07-01T00:00:00.000Z': 0.0640325808,
    u'2005-08-01T00:00:00.000Z': 0.0758995467,
    u'2005-09-01T00:00:00.000Z': 0.0693853782,
    u'2005-10-01T00:00:00.000Z': 0.0677155855,

In [15]:
def generate_time_series_jsons(
    ts_dict, jsons_dir="/Users/abenassi/github/series-tiempo/catalogo/datos/series/"):

    for series_id, value in ts_dict.iteritems():
        file_name = "{}.json".format(series_id)
        
        with open(os.path.join(jsons_dir, file_name), "wb") as f:
            json.dump(value, f)

In [20]:
ts_dict

{'11.3_VIPAA_2004_M_31': [(u'var_igual_periodo_anio_anterior',
   {u'2004-01-01T00:00:00.000Z': None,
    u'2004-02-01T00:00:00.000Z': None,
    u'2004-03-01T00:00:00.000Z': None,
    u'2004-04-01T00:00:00.000Z': None,
    u'2004-05-01T00:00:00.000Z': None,
    u'2004-06-01T00:00:00.000Z': None,
    u'2004-07-01T00:00:00.000Z': None,
    u'2004-08-01T00:00:00.000Z': None,
    u'2004-09-01T00:00:00.000Z': None,
    u'2004-10-01T00:00:00.000Z': None,
    u'2004-11-01T00:00:00.000Z': None,
    u'2004-12-01T00:00:00.000Z': None,
    u'2005-01-01T00:00:00.000Z': 0.0631114207,
    u'2005-02-01T00:00:00.000Z': 0.0657826605,
    u'2005-03-01T00:00:00.000Z': 0.0861095782,
    u'2005-04-01T00:00:00.000Z': 0.1362787407,
    u'2005-05-01T00:00:00.000Z': 0.1527559219,
    u'2005-06-01T00:00:00.000Z': 0.1025891554,
    u'2005-07-01T00:00:00.000Z': 0.0640325808,
    u'2005-08-01T00:00:00.000Z': 0.0758995467,
    u'2005-09-01T00:00:00.000Z': 0.0693853782,
    u'2005-10-01T00:00:00.000Z': 0.0677155855,

In [21]:
generate_time_series_jsons(ts_dict)

In [27]:
import json

In [28]:
a = json.load(df.to_json(orient="columns", date_format="iso"))

In [ ]:
get_field_metadata()

In [84]:
df = get_time_series_df(ied_ts_ids)

In [85]:
df

,var_igual_periodo_anio_anterior,eph_continua_tasa_desempleo_total,tasas_interes_badlar,tipo_cambio_real_multilateral
indice_tiempo,,,,
1991-02-01,NaN,NaN,NaN,177.447682
1991-03-01,NaN,NaN,NaN,156.765857
1991-04-01,NaN,NaN,NaN,151.500774
1991-05-01,NaN,NaN,NaN,147.402654
1991-06-01,NaN,NaN,NaN,141.025126
1991-07-01,NaN,NaN,NaN,138.204629
1991-08-01,NaN,NaN,NaN,138.993119
1991-09-01,NaN,NaN,NaN,138.933196
1991-10-01,NaN,NaN,NaN,134.533915


In [138]:
df["oferta_global_pbi_precios_2011_II"] = df.oferta_global_pbi / (df.indice_precios_implicito_pbi_mercado / df.indice_precios_implicito_pbi_mercado["2011-04-01"])

In [128]:
df["demanda_global_ibif_total / ofert_global_pbi"] = df["demanda_global_ibif_total"] / df["oferta_global_pbi"]

In [141]:
df.to_csv("/Users/abenassi/github/series-tiempo/ejemplos/deflacion_serie_por_otra.csv", encoding="utf8")

In [142]:
ipc = get_time_series_df("104.1_I2NG_2016_M_22")

## Upload data y metadata

## Setup

In [28]:
import pysftp
import requests
import yaml
import os

In [2]:
cnopts = pysftp.CnOpts()
cnopts.hostkeys.load('/Users/abenassi/.ssh/known_hosts')
cnopts.hostkeys = None 

## Main steps

In [25]:
def upload_file_to_ind(local_path, remote_dir=None, file_name=None, config_ind="ind/config_ind.yaml"):
    """Carga de un archivo a la infraestructura."""

    if isinstance(config_ind, dict):
        ind_params = config_ind
    else:
        with open(config_ind, 'r') as f:
            ind_params = yaml.load(f)
        
    remote_dir = remote_dir or '/home/{}'.format(ind_params["user"])

    with pysftp.Connection(ind_params["host"], username=ind_params["user"], password=ind_params["pass"], cnopts=cnopts) as sftp:
        with sftp.cd(remote_dir):
            if file_name:
                remote_path = os.path.join(remote_dir, file_name)
                sftp.put(local_path, remote_path)
            else:
                return sftp.put(local_path)


In [23]:
def upload_datajson_to_ind(local_path, config_ind_path="ind/config_ind.yaml"):
    
    with open(config_ind_path, 'r') as f:
        ind_params = yaml.load(f)
        
    upload_file_to_ind(local_path, config_ind=ind_params)

    headers = {"Authorization": ind_params["api"]["metadata"]["auth_header"]}
    url = ind_params["api"]["metadata"]["url"]
    
    return requests.get(url, headers=headers)

In [24]:
res_api = upload_datajson_to_ind('/Users/abenassi/github/series-tiempo/catalogo/datos/data.json')

In [21]:
res_api_metadata = upload_datajson_to_ind('/Users/abenassi/Downloads/data.json')

In [22]:
res_api_metadata.content

'<webresult>\n  <result>OK</result>\n  <message>Job started</message>\n  <id>678ed0bb-c312-4c2e-b4d6-c98932025288</id>\n</webresult>\n\n'

In [38]:
def upload_distribution_to_ind(local_path, distribution_id, config_ind_path="ind/config_ind.yaml"):
    
    with open(config_ind_path, 'r') as f:
        ind_params = yaml.load(f)
        
    file_name = "{}.csv".format(distribution_id)
    upload_file_to_ind(local_path, file_name=file_name, config_ind=ind_params)

    headers = {"Authorization": ind_params["api"]["data"]["auth_header"]}
    url = ind_params["api"]["data"]["url"].format(distribution_id)
    print(url)
    
    return requests.get(url, headers=headers)

In [44]:
upload_file_to_ind("/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/1/1.1.csv", file_name="73.csv")

In [45]:
res_api_data = upload_distribution_to_ind("/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/1/1.1.csv", 73)

http://192.168.150.211:9080/pentaho-di/kettle/runJob/?job=public%2Fmetadata-dinamica%2Fjb_invocacion_api_ext&ID_DISTRIBUTION=73&TOKEN=YWJlbmFzc2k6OjokMmEkMTAkWkJvOHJEOG9nbkJDSDB3dGFHREYxLkVLcVdoWUxPQzc1QXh3UDQxcFRtOFB4ZUVZdkdqNm0=


In [46]:
res_api_data.content

'<webresult>\n  <result>OK</result>\n  <message>Job started</message>\n  <id>f6165dc0-8c97-4ec3-9da7-4e1e310b31a8</id>\n</webresult>\n\n'

In [39]:
with pysftp.Connection(host, username=user, password=password) as sftp:
    with sftp.cd('/home/abenassi'):             # temporarily chdir to public
        sftp.put('/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/1/63.csv')  # upload file to public/ on remote
        # sftp.get('remote_file')         # get a remote file

In [41]:
url = url_api_ind
headers = {"Authorization": auth_header}
res = requests.get(url, headers=headers)

In [42]:
res.content

'<webresult>\n  <result>OK</result>\n  <message>Job started</message>\n  <id>1ce9bc81-e519-4f7b-b921-ac2c1473e84f</id>\n</webresult>\n\n'

### Pruebas y chequeos

In [110]:
from pydatajson.readers import read_catalog
from pydatajson import DataJson

In [111]:
dj = DataJson("catalog.json", "/Users/abenassi/github/series-tiempo/catalogo/codigo/schemas/")

In [112]:
dj.is_valid_catalog("/Users/abenassi/github/series-tiempo/catalogo/datos/data.json")

True

In [113]:
catalog = read_catalog("/Users/abenassi/github/series-tiempo/catalogo/datos/data.json")

In [7]:
catalog[0]["title"]

u'Datos Programaci\xf3n Macroecon\xf3mica'

### Chequeos webdav

In [5]:
import requests
from requests.auth import HTTPBasicAuth

In [6]:
res = requests.get("https://meconcd.mecon.gob.ar/remote.php/webdav/", 
                   auth=HTTPBasicAuth(options["webdav_login"], options["webdav_password"]), verify=False)

/Users/abenassi/anaconda/envs/series-tiempo/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [9]:
import pycurl

In [32]:
res.

<RequestsCookieJar[Cookie(version=0, name='oc41be7089f3', value='sj0jiee5lnk1pov11e98rs6vg7', port=None, port_specified=False, domain='meconcd.mecon.gob.ar', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [12]:
import easywebdav

In [21]:
webdav = easywebdav.connect('meconcd.mecon.gob.ar/remote.php/webdav/', username='abenassi', password='alpha1g13', protocol='http')

In [22]:
webdav.mkdir('some_dir')

OperationFailed: Failed to create directory "some_dir".
  Operation     :  MKCOL some_dir
  Expected code :  201 Created
  Actual code   :  302 Found

In [10]:
import webdav.client as wc
options = {
 'webdav_hostname': "https://meconcd.mecon.gob.ar/remote.php/webdav/",
 'webdav_login':    "abenassi",
 'webdav_password': "alpha1g13",
  "cert_path": "/Users/abenassi/github/series-tiempo/catalogo/codigo/certificate/AutoridadCertificantedelMinisteriodeEconomiaversionII.crt",
    
 
}
client = wc.Client(options)
client.list()


NotConnection: Not connection with https://meconcd.mecon.gob.ar/remote.php/webdav/

In [14]:
client.mkdir("new_dir")

RemoteParentNotFound: Remote parent for: /new_dir/ not found

In [12]:
client.clean("/")

In [9]:
client.upload_sync(remote_path="helpers.py", local_path="helpers.py")

RemoteParentNotFound: Remote parent for: /helpers.py not found